In [3]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

In [4]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/12 15:03:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/12 15:03:13 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [5]:
import pandas as pd

# Sample data for the SurveyLog table
survey_log_data = {
    'id': [5, 5, 5, 5],
    'action': ['show', 'answer', 'show', 'skip'],
    'question_id': [285, 285, 369, 369],
    'answer_id': [None, 124124, None, None],
    'q_num': [1, 1, 2, 2],
    'timestamp': [123, 124, 125, 126]
}

# Creating a pandas DataFrame for SurveyLog table
survey_log_df = pd.DataFrame(survey_log_data)

# Displaying the SurveyLog DataFrame
print("SurveyLog table:")
print(survey_log_df)



df_person = spark.createDataFrame(survey_log_df)
df_person.createOrReplaceTempView("SurveyLog")



SurveyLog table:
   id  action  question_id  answer_id  q_num  timestamp
0   5    show          285        NaN      1        123
1   5  answer          285   124124.0      1        124
2   5    show          369        NaN      2        125
3   5    skip          369        NaN      2        126


In [6]:
query = """
    select
        question_id,
        sum(
            case
                when action = 'answer' then 1
                else 0
            end
        ) / sum(
            case
                when action = 'show' then 1
                else 0
            end
        ) as answer_rate
    from SurveyLog s
    group by question_id
    order by answer_rate desc, question_id asc
    limit 1
"""

In [22]:
query = """
select question_id,sum(

    case
        when action='answer' then 1
        else 0
    end
)/ sum (
    case
        when action='show' then 1
        else 0
    end
) as percent
from SurveyLog
group by question_id
order by percent desc
limit 1

"""

In [23]:
spark.sql(query).show()

+-----------+-------+
|question_id|percent|
+-----------+-------+
|        285|    1.0|
+-----------+-------+

